In [37]:
import warnings ##忽略警告信息
warnings.filterwarnings('ignore')
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
path_prefix = 'E:\\BaiduNetdiskDownload\\数据\\hw4\\'

In [39]:
def load_training_data(path='E:\\BaiduNetdiskDownload\\数据\\hw4\\training_nolabel.txt'):
    # 把training时需要的data读进来
    # 如果是'training_label.txt'，需要读取label，如果是'training_nolabel.txt'，不需要读取label
    if 'training_label' in path:
        with open(path, 'r', encoding='UTF-8') as f:
            lines = f.readlines()  ##依次读取每行
            lines = [line.strip('\n').split(' ') for line in lines]  ##移除每行头尾的换行符并以空格为分隔符
        x = [line[2:] for line in lines]    ##对于每行的每个词，都从第三个字符开始提取
        y = [line[0] for line in lines]     ##对于每行的每个词，只提取第一个字符
        return x, y
    else:
        with open(path, 'r', encoding='UTF-8') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='E:\\BaiduNetdiskDownload\\数据\\hw4\\testing_data.txt'):
    # 把testing时需要的data读进来
    with open(path, 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]#将test_data分成许多行，从第二行开始，去除每行首尾的换行符并以“，”号
                                           #将每行分成许多个小部分，并用“”将各个小部分连接起来
        X = [sen.split(' ') for sen in X]  ##将上述处理好的文本的每行以空格符为标志将每行分割成若干小部分。
    return X ##返回的是一个二维的list

def evaluation(outputs, labels):
    #outputs => probability (float)
    #labels => labels
    outputs[outputs>=0.5] = 1 # 大于等于0.5为有恶意
    outputs[outputs<0.5] = 0 # 小于0.5为无恶意
    correct = torch.sum(torch.eq(outputs, labels)).item()##torch.eq()函数比较两个tensor是否相等，相等的地方写1，不相等的地方写0
    return correct

<font size=7 face="黑体">Train Word To Vector:</font>

In [ ]:
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec
import gensim

In [40]:
def train_word2vec(x):
    # 训练 word to vector 的 word embedding
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)##x为输入语料集，size为特征向量的维度， 
    #window：表示当前词与预测词在一个句子中的最大距离是多少 ，min_count: 可以对字典做截断. 词频少于min_count次数的单词会被丢弃掉, 默认值为5 
    #workers参数控制训练的并行数， iter： 迭代次数， sg： 用于设置训练算法，默认为0，对应CBOW算法；sg=1则采用skip-gram算法
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data('E:\\BaiduNetdiskDownload\\数据\\hw4\\training_label.txt')
    train_x_no_label = load_training_data('E:\\BaiduNetdiskDownload\\数据\\hw4\\training_nolabel.txt')

    print("loading testing data ...")
    test_x = load_testing_data('E:\\BaiduNetdiskDownload\\数据\\hw4\\testing_data.txt')

    model = train_word2vec(train_x + train_x_no_label + test_x)
    
    print("saving model ...")
    # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
    model.save(os.path.join(path_prefix, 'w2v_all.model'))

loading training data ...
loading testing data ...
saving model ...


<font size=7 face="黑体">Data Preprocess:</font>